In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data = datagen.flow_from_directory("D:/datasets/PlantVillage/train", target_size=(128, 128), batch_size=32, class_mode='categorical', subset='training')
val_data = datagen.flow_from_directory("D:/datasets/PlantVillage/train", target_size=(128, 128), batch_size=32, class_mode='categorical', subset='validation')

Found 231 images belonging to 1 classes.
Found 57 images belonging to 1 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=5, validation_data=val_data)

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\indum\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


C:\Users\indum\anaconda3\Lib\site-packages\keras\src\ops\nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\indum\anaconda3\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 944ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 890ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 848ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 917ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [7]:
model.export("plant_disease_model")

INFO:tensorflow:Assets written to: plant_disease_model\assets


INFO:tensorflow:Assets written to: plant_disease_model\assets


Saved artifact at 'plant_disease_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1821326863312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326864080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326864848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326864656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326860240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326865424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326862736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1821326867152: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [23]:
!gsutil cp -r plant_disease_model gs://your-bucket-name/

'gsutil' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
!gcloud ai models upload \
  --region=us-central1 \
  --display-name=plant-disease-model \
  --artifact-uri=gs://your-bucket-name/plant_disease_model \
  --container-image-uri=gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-12:latest

'gcloud' is not recognized as an internal or external command,
operable program or batch file.


In [29]:
import requests, json
from tensorflow.keras.preprocessing import image
import numpy as np

In [31]:
img = image.load_img("D:/image/animal.jpg", target_size=(128, 128)) 
img_array = image.img_to_array(img) / 255.0
preprocessed_image = np.expand_dims(img_array, axis=0)

In [33]:
payload = json.dumps({"instances": preprocessed_image.tolist()})
headers = {"Content-Type": "application/json", "Authorization": "Bearer YOUR_ACCESS_TOKEN"}

In [35]:
url = "https://us-central1-prediction-aiplatform.googleapis.com/v1/projects/your-project-id/locations/us-central1/endpoints/your-endpoint-id:predict"  # from GCP
response = requests.post(url, headers=headers, data=payload)
print("Prediction:", response.json())

Prediction: {'error': {'code': 401, 'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_TYPE_UNSUPPORTED', 'metadata': {'method': 'google.cloud.aiplatform.v1.PredictionService.Predict', 'service': 'aiplatform.googleapis.com'}}]}}
